In [1]:
import asyncio
import aiohttp
import pandas as pd
import nest_asyncio
nest_asyncio.apply()
from dotenv import load_dotenv
load_dotenv()
import os

# Socrata app token 
APP_TOKEN = os.getenv("CHICAGO_API_TOKEN")

# Endpoint and config
BASE_URL = "https://data.cityofchicago.org/resource/ijzp-q8t2.json"
LIMIT = 10000
CONCURRENT_REQUESTS = 5

semaphore = asyncio.Semaphore(CONCURRENT_REQUESTS)

In [2]:
async def fetch_batch(session, offset, retries=5):
    headers = {"X-App-Token": APP_TOKEN}
    params = {
        "$select": "date",  # Changed to fetch the 'date' column
        "$limit": LIMIT,
        "$offset": offset,
        "$where": "date IS NOT NULL"
    }
    async with semaphore:
        for attempt in range(retries):
            try:
                async with session.get(BASE_URL, headers=headers, params=params) as resp:
                    if resp.status == 200:
                        return await resp.json()
                    elif resp.status in [429, 500, 502, 503, 504]:
                        wait = 2 ** attempt
                        print(f"Retryable error {resp.status} at offset {offset}. Retrying in {wait}s...")
                        await asyncio.sleep(wait)
                    else:
                        print(f"Non-retryable error at offset {offset}: HTTP {resp.status}")
                        return []
            except Exception as e:
                print(f"Exception at offset {offset}: {e}")
        return []

async def fetch_all_data():
    all_records = []
    offset = 0

    async with aiohttp.ClientSession() as session:
        while True:
            tasks = [fetch_batch(session, offset + i * LIMIT) for i in range(CONCURRENT_REQUESTS)]
            results = await asyncio.gather(*tasks)

            batch_records = [record for batch in results if batch for record in batch]
            all_records.extend(batch_records)

            print(f"Retrieved: {len(all_records)} records...")

            if any(len(batch) < LIMIT for batch in results):
                break

            offset += CONCURRENT_REQUESTS * LIMIT

    # Convert to DataFrame and parse 'date' column
    df = pd.DataFrame(all_records)
    df["date"] = pd.to_datetime(df["date"], errors="coerce")
    return df

# ✅ Run this inside an async context (e.g., Jupyter cell)
df = await fetch_all_data()
print(f"Total records: {len(df)}")
df.head()


Retrieved: 50000 records...
Retrieved: 100000 records...
Retrieved: 150000 records...
Retrieved: 200000 records...
Retrieved: 250000 records...
Retrieved: 300000 records...
Retrieved: 350000 records...
Retrieved: 400000 records...
Retrieved: 450000 records...
Retrieved: 500000 records...
Retrieved: 550000 records...
Retrieved: 600000 records...
Retrieved: 650000 records...
Retrieved: 700000 records...
Retrieved: 750000 records...
Retrieved: 800000 records...
Retrieved: 850000 records...
Retrieved: 900000 records...
Retrieved: 950000 records...
Retrieved: 1000000 records...
Retrieved: 1050000 records...
Retrieved: 1100000 records...
Retrieved: 1150000 records...
Retrieved: 1200000 records...
Retrieved: 1250000 records...
Retrieved: 1300000 records...
Retrieved: 1350000 records...
Retrieved: 1400000 records...
Retrieved: 1450000 records...
Retrieved: 1500000 records...
Retrieved: 1550000 records...
Retrieved: 1600000 records...
Retrieved: 1650000 records...
Retrieved: 1700000 records...


,date
0,2022-07-29 03:39:00
1,2023-01-03 16:44:00
2,2020-08-10 09:45:00
3,2017-08-26 10:00:00
4,2023-09-06 17:00:00


In [ ]:
# Ensure date is in datetime format
df['date'] = pd.to_datetime(df['date'], errors='coerce')

# Drop rows with missing dates
df = df.dropna(subset=['date'])

# Extract hour and year
df['hour'] = df['date'].dt.hour
df['year'] = df['date'].dt.year

# ✅ Filter to years 2010–2019
df = df[(df['year'] >= 2010) & (df['year'] <= 2019)]

# Define time ranges using hour
def get_time_range(hour):
    if 4 <= hour < 7:
        return "4am–7am"
    elif 7 <= hour < 12:
        return "7am–12pm"
    elif 12 <= hour < 17:
        return "12pm–5pm"
    elif 18 <= hour < 21:
        return "6pm–9pm"
    elif 21 <= hour <= 23:
        return "9pm–12am"
    else:
        return "12am–4am"

# Apply time range function
df['time_range'] = df['hour'].apply(get_time_range)

# Group by time range and year
time_year_counts = df.groupby(['time_range', 'year']).size().reset_index(name='count')

# Optional: Sort for readability
time_range_order = ["12am–4am", "4am–7am", "7am–12pm", "12pm–5pm", "6pm–9pm", "9pm–12am"]
time_year_counts['time_range'] = pd.Categorical(time_year_counts['time_range'], categories=time_range_order, ordered=True)
time_year_counts = time_year_counts.sort_values(['year', 'time_range'])

# Show result
print(time_year_counts.head())

# ✅ Export to CSV
time_year_counts.to_csv("time_range_counts_2010_2019.csv", index=False)
print("Exported to 'time_range_counts_2010_2019.csv'")